In [0]:
import urllib
import requests
import pandas as pd
import numpy as np
import re
import datetime
import random
import os
import sys
import shutil
import json

# Config file name and path

In [0]:
fkr30k_config = "C:/Users/Vishal/Desktop/AIML/Capstone/flkr30k_config.txt"
fkr8k_config = "C:/Users/Vishal/Desktop/AIML/Capstone/flkr8k_config.txt"
GCC_config = "C:/Users/Vishal/Desktop/AIML/Capstone/cc_config.txt"
mscoco_config = "C:/Users/Vishal/Desktop/AIML/Capstone/mscoco_config.txt"

In [0]:
## Base Function to extract dataset
def data_extracter():
    if dataset == "fkr30k":
        caption_header = "comment"
        imagename_header = "image_name"
        for data,count in dict.items():
            flkr30k_process(data,count)
    elif dataset == "fkr8k":
        caption_header = "comment"
        imagename_header = "image_name"
        for data,count in dict.items():
            flkr8k_process(data,count)
    elif dataset == "GCC":
        caption_header = "comment"
        imagename_header = "URL"
        for data,count in dict.items():
            GCC_process(data,count)


In [0]:
## Function to extract data from flickr30k with filters in config file
def flkr30k_process_filter(data,count):
    df = pd.read_csv(src_Caption_filename,sep='|')
    print(df.head())
    print(df.columns)
    df.rename(columns=lambda x: x.strip(), inplace = True)
    print(df.columns)
    df[caption_header] = df[caption_header].fillna(value="No Value")
    df_filter = df[df[caption_header].str.contains(data)]
    image_names = df_filter[imagename_header].unique()
    if image_names.size <= int(count):
        copy_files(image_names)
        create_keycaptions(df_filter,image_names)
    else:
        copy_files(image_names[:int(count)])
        create_keycaptions(df_filter,image_names[:int(count)])
    

In [0]:
## Function to fectch 8k data (Count parameter) from flickr30k
def flkr30k_process(data,count):
    df = pd.read_csv(src_Caption_filename,sep='|')
    print(df.head())
    print(df.columns)
    df.rename(columns=lambda x: x.strip(), inplace = True)
    print(df.columns)
    df[caption_header] = df[caption_header].fillna(value="No Value")
    #df_filter = df[df[caption_header].str.contains(data)]
    image_names = df[imagename_header].unique()
    copy_files(image_names[:int(count)])
    create_keycaptions(df,image_names[:int(count)])

In [0]:
## Function to fetch 8k data from MSCoco
def mscoco_process(data,count):
    df = pd.read_csv(src_Caption_filename,sep="|")
    print(df.head())
    df.columns = ['image_name','comment']
    #df['image_name'] = df.apply(lambda x: x['image_name'][:-2], axis = 1)
    #print(df.head())
    #df_filter = df[df[caption_header].str.contains(data)]
    image_names = df[imagename_header].unique()
    copy_files(image_names[:int(count)])
    create_keycaptions(df,image_names[:int(count)])

In [0]:
## Function to fetch subset based on config file filter from flkr8k dataset
def flkr8k_process_filter(data,count):
    df = pd.read_csv(src_Caption_filename,sep="\t")
    print(df.head())
    df.columns = ['image_name','comment']
    df['image_name'] = df.apply(lambda x: x['image_name'][:-2], axis = 1)
    print(df.head())
    df_filter = df[df[caption_header].str.contains(data)]
    image_names = df_filter[imagename_header].unique()
    if image_names.size <= int(count):
        copy_files(image_names)
        create_keycaptions(df_filter,image_names)
    else:
        copy_files(image_names[:int(count)])
        create_keycaptions(df_filter,image_names[:int(count)])

In [0]:
## Function to fetch subset based on (count) from flkr8k dataset
def flkr8k_process(data,count):
    df = pd.read_csv(src_Caption_filename,sep="\t")
    print(df.head())
    df.columns = ['image_name','comment']
    df['image_name'] = df.apply(lambda x: x['image_name'][:-2], axis = 1)
    image_names = df[imagename_header].unique()
    copy_files(image_names[:int(count)])
    create_keycaptions(df,image_names[:int(count)])

In [0]:
## Function to fetch subset based on (count) from MSCoco dataset
def mscoco_process(data,count):
    df = pd.read_csv(src_Caption_filename,sep="|")
    print(df.head())
    df.columns = ['image_name','comment']
    #df['image_name'] = df.apply(lambda x: x['image_name'][:-2], axis = 1)
    #print(df.head())
    #df_filter = df[df[caption_header].str.contains(data)]
    image_names = df[imagename_header].unique()
    copy_files(image_names[:int(count)])
    create_keycaptions(df,image_names[:int(count)])

In [0]:
## Function to fetch data from Google Conceptual Captions
def GCC_process(count):
    df = pd.read_csv(src_Caption_filename,sep="\t")
    print(df.head())
    df.columns = ['comment','URL']
   # df_filter = df[df[caption_header].str.contains(data)]
    df_filter2 = df.head(int(count))
    print(df_filter2.head())
    download_GCC(df_filter2)

In [0]:
## Base Function to download data from GCC URL
def download_GCC(dtest):
    print("Calling download GCC")
    for (idx, row) in dtest.iterrows():
        caption = row.loc['comment']
        url = row.loc['URL']
        temp = re.sub(r"\s+", "", caption)
        now = datetime.datetime.now()
        # key = temp[0:9] + str(now.strftime("%y%m%d%H%M%S")) + str(random.randrange(100,999,3))
        key = str(now.strftime("%y%m%d%H%M%S")) + str(random.randrange(100,9999,3))
    
        download_img(url,key,caption)

In [0]:
counter = 0

In [0]:
## Download image and create key caption file
def download_img(url,key,caption):
    try:
        global counter
        print("Downloading image from URL " + url)
        resource = urllib.request.urlopen(url)
        output = open(destination_folder + '/' + "GCC" + key + ".jpg","wb")
        output.write(resource.read())
        output.close()
        print("Download Complete")
        write_keyCaptions(key + ".jpg",caption)
        counter = counter + 1
        print(counter)
    except Exception as e:
        print("Error while downloading due to : ",e)
        with open(destination_folder + '/' + "Error_URL.txt",'a') as f:
            f.write(url+"\n")
            f.close()

In [0]:
## Function to copy files from base to destination folder
def copy_files(image_names):
    for item in image_names:
        print("Image copying == ",item)
        file_path = os.path.join(src_image_folder, item)
        print(file_path)
        if os.path.isfile(file_path):
            shutil.copy(file_path, destination_folder + "/" + dataset + item)
        else:
            print("ERROR in copying image files")


In [0]:
## Function to output the key captions to "Key_caption.txt"
def write_keyCaptions(key,caption):
    with open(str(destination_folder) + "/key_caption.txt",'a') as f:
                f.write(dataset+key + " | "+caption+"\n")
                f.close()

In [0]:
## Function to create key captions file
def create_keycaptions(df,image_names):
    for x in image_names:
        df2 = df[df[imagename_header].str.contains(x)]
        for (idx, row) in df2.iterrows():
            caption = row.loc[caption_header]
            image_name = row.loc[imagename_header]
            write_keyCaptions(image_name,caption)

In [0]:
## Function to read config file
def read_config(config_file):
    with open(config_file,'r') as f:
            temp = f.readlines()
            f.close()
    return temp

In [0]:
dict = {}

In [0]:
## Function to extract parameter values from config file
def extract_config(temp):
    for x in temp:
        x2 = x.split('=')
        if x2[0].strip() == 'dataset':
            dataset = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'src_Caption_filename':
            src_Caption_filename = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'src_image_folder':
            src_image_folder = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'destination_folder':
            destination_folder = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'Data:':
            pass
            Filter = True
        else:
            x3 = x2[0].split('|')
            dict[x3[0].strip()] = x3[1].strip()
            print(x3[0].strip())
            print(x3[1].strip())

In [0]:
## Extract data from MSCOCO json file

file_name = 'C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/annotations/captions_train2017.json'
with open(file_name) as f:
    data = json.load(f)
    
annot = data['annotations']
file_text=""
file1 = open("C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/annotations/MSCOCO_TRAIN_captions6.txt","a")

for i in annot:
    img_id = i['image_id']
    cap_id = i['id']
    cap = i['caption']
    temp_text = str('%0.12d' % img_id)+".jpg"+"|" +cap + "\n"
    file1.write(temp_text)
    #file_text = file_text + temp_text
    print(temp_text)
#file1 = open("C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/annotations/MSCOCO_TRAIN_captions5.txt","w")
#file1.write(file_text)
file1.close
    

# Execute from here

## To Create flkr8k and flkr30k subset

In [0]:
with open(fkr8k_config,'r') as f:
            temp = f.readlines()
            f.close()

In [0]:
for x in temp:
        x2 = x.split('=')
        if x2[0].strip() == 'dataset':
            dataset = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'src_Caption_filename':
            src_Caption_filename = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'src_image_folder':
            src_image_folder = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'destination_folder':
            destination_folder = x2[1].strip()
            print(x2[1].strip())

fkr8k
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_text/Flickr8k.token.txt
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/subset


In [0]:
if dataset == "fkr30k":
        caption_header = "comment"
        imagename_header = "image_name"
        for data,count in dict.items():
            flkr30k_process(data,count)
elif dataset == "fkr8k":
        caption_header = "comment"
        imagename_header = "image_name"
        count=8000
        flkr8k_process(data,count)
        #for data,count in dict.items():
        #    flkr8k_process(data,count)
elif dataset == "GCC":
        caption_header = "comment"
        imagename_header = "URL"
        for data,count in dict.items():
            GCC_process(data,count)
elif dataset == "mscoco":
        caption_header = "comment"
        imagename_header = "image_name"
        count=8000
        mscoco_process(data,count)
    

   1000268201_693b08cb0e.jpg#0  \
0  1000268201_693b08cb0e.jpg#1   
1  1000268201_693b08cb0e.jpg#2   
2  1000268201_693b08cb0e.jpg#3   
3  1000268201_693b08cb0e.jpg#4   
4  1001773457_577c3a7d70.jpg#0   

  A child in a pink dress is climbing up a set of stairs in an entry way .  
0              A girl going into a wooden building .                        
1   A little girl climbing into a wooden playhouse .                        
2  A little girl climbing the stairs to her playh...                        
3  A little girl in a pink dress going into a woo...                        
4         A black dog and a spotted dog are fighting                        
Image copying ==  1000268201_693b08cb0e.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1000268201_693b08cb0e.jpg
Image copying ==  1001773457_577c3a7d70.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1001773457_577c3a7d70.jpg
Image copyi

Image copying ==  106514190_bae200f463.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\106514190_bae200f463.jpg
Image copying ==  1067180831_a59dc64344.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1067180831_a59dc64344.jpg
Image copying ==  1067675215_7336a694d6.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1067675215_7336a694d6.jpg
Image copying ==  1067790824_f3cc97239b.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1067790824_f3cc97239b.jpg
Image copying ==  1072153132_53d2bb1b60.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1072153132_53d2bb1b60.jpg
Image copying ==  107318069_e9f2ef32de.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\107318069_e9f2ef32de.jpg
Image copying ==  1075716537_62105738b

Image copying ==  1115565519_d976d4b1f1.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1115565519_d976d4b1f1.jpg
Image copying ==  1115679311_245eff2f4b.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1115679311_245eff2f4b.jpg
Image copying ==  111766423_4522d36e56.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\111766423_4522d36e56.jpg
Image copying ==  1117972841_2b9261f95f.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1117972841_2b9261f95f.jpg
Image copying ==  1118557877_736f339752.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1118557877_736f339752.jpg
Image copying ==  1119015538_e8e796281e.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1119015538_e8e796281e.jpg
Image copying ==  1119418776_58e4b93

Image copying ==  1191338263_a4fa073154.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1191338263_a4fa073154.jpg
Image copying ==  1193116658_c0161c35b5.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1193116658_c0161c35b5.jpg
Image copying ==  119534510_d52b3781a3.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\119534510_d52b3781a3.jpg
Image copying ==  1197800988_7fb0ca4888.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1197800988_7fb0ca4888.jpg
Image copying ==  1198194316_543cc7b945.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1198194316_543cc7b945.jpg
Image copying ==  1204996216_71d7519d9a.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1204996216_71d7519d9a.jpg
Image copying ==  1206506157_c7956ac

Image copying ==  1258913059_07c613f7ff.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1258913059_07c613f7ff.jpg
Image copying ==  1259936608_e3f0064f23.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1259936608_e3f0064f23.jpg
Image copying ==  1260816604_570fc35836.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1260816604_570fc35836.jpg
Image copying ==  1262077938_8b9516c273.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1262077938_8b9516c273.jpg
Image copying ==  1262454669_f1caafec2d.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1262454669_f1caafec2d.jpg
Image copying ==  1262583859_653f1469a9.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\1262583859_653f1469a9.jpg
Image copying ==  1263126002_881eb

...

Image copying ==  559102835_472ff702b5.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\559102835_472ff702b5.jpg
Image copying ==  560278886_df4bd2e212.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\560278886_df4bd2e212.jpg
Image copying ==  561179884_8b6b925ef9.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\561179884_8b6b925ef9.jpg
Image copying ==  561179890_af8e31cb2e.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\561179890_af8e31cb2e.jpg
Image copying ==  561417861_8e25d0c0e8.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\561417861_8e25d0c0e8.jpg
Image copying ==  561940436_64d6fc125d.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\561940436_64d6fc125d.jpg
Image copying ==  562588230_edb2c071c8.jpg
C:/

Image copying ==  616045808_0286d0574b.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\616045808_0286d0574b.jpg
Image copying ==  616177206_0e16c33f6b.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\616177206_0e16c33f6b.jpg
Image copying ==  617038406_4092ee91dd.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\617038406_4092ee91dd.jpg
Image copying ==  618292739_0fdc2ccab0.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\618292739_0fdc2ccab0.jpg
Image copying ==  618771382_459bb492e4.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\618771382_459bb492e4.jpg
Image copying ==  619169586_0a13ee7c21.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\619169586_0a13ee7c21.jpg
Image copying ==  621000329_84f48948eb.jpg
C:/

Image copying ==  696663662_232edd58af.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\696663662_232edd58af.jpg
Image copying ==  69710411_2cf537f61f.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\69710411_2cf537f61f.jpg
Image copying ==  69710415_5c2bfb1058.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\69710415_5c2bfb1058.jpg
Image copying ==  697490420_67d8d2a859.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\697490420_67d8d2a859.jpg
Image copying ==  697582336_601462e052.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\697582336_601462e052.jpg
Image copying ==  697778778_b52090709d.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\697778778_b52090709d.jpg
Image copying ==  698107542_3aa0ba78b4.jpg
C:/User

Image copying ==  757332692_6866ae545c.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\757332692_6866ae545c.jpg
Image copying ==  758921886_55a351dd67.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\758921886_55a351dd67.jpg
Image copying ==  759015118_4bd3617e60.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\759015118_4bd3617e60.jpg
Image copying ==  760138567_762d9022d4.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\760138567_762d9022d4.jpg
Image copying ==  760180310_3c6bd4fd1f.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\760180310_3c6bd4fd1f.jpg
Image copying ==  762947607_2001ee4c72.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\762947607_2001ee4c72.jpg
Image copying ==  763577068_4b96ed768b.jpg
C:/

Image copying ==  836768303_d748df5546.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\836768303_d748df5546.jpg
Image copying ==  836828001_af98d16256.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\836828001_af98d16256.jpg
Image copying ==  837893113_81854e94e3.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\837893113_81854e94e3.jpg
Image copying ==  837919879_94e3dacd83.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\837919879_94e3dacd83.jpg
Image copying ==  838074897_9d6270b3cd.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\838074897_9d6270b3cd.jpg
Image copying ==  839295615_bb9baf2f95.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset\839295615_bb9baf2f95.jpg
Image copying ==  842960985_91daf0d6ec.jpg
C:/

## To create mscoco subset

In [0]:
with open(mscoco_config,'r') as f:
            temp = f.readlines()
            f.close()

In [0]:
for x in temp:
        x2 = x.split('=')
        if x2[0].strip() == 'dataset':
            dataset = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'src_Caption_filename':
            src_Caption_filename = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'src_image_folder':
            src_image_folder = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'destination_folder':
            destination_folder = x2[1].strip()
            print(x2[1].strip())

mscoco
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/annotations/MSCOCO_TRAIN_captions6.txt
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/subset


In [0]:
if dataset == "fkr30k":
        caption_header = "comment"
        imagename_header = "image_name"
        for data,count in dict.items():
            flkr30k_process(data,count)
elif dataset == "fkr8k":
        caption_header = "comment"
        imagename_header = "image_name"
        count=8000
        flkr8k_process(data,count)
        #for data,count in dict.items():
        #    flkr8k_process(data,count)
elif dataset == "GCC":
        caption_header = "comment"
        imagename_header = "URL"
        for data,count in dict.items():
            GCC_process(data,count)
elif dataset == "mscoco":
        caption_header = "comment"
        imagename_header = "image_name"
        count=8000
        mscoco_process(data,count)

   000000203564.jpg A bicycle replica with a clock as the front wheel.
0  000000322141.jpg  A room with blue walls and a white sink and door.
1  000000016977.jpg  A car that seems to be parked illegally behind...
2  000000106140.jpg  A large passenger airplane flying through the ...
3  000000106140.jpg  There is a GOL plane taking off in a partly cl...
4  000000322141.jpg   Blue and white color scheme in a small bathroom.
Image copying ==  000000322141.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000322141.jpg
Image copying ==  000000016977.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000016977.jpg
Image copying ==  000000106140.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000106140.jpg
Image copying ==  000000203564.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000203564.jpg
Image copying ==  000000571635.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/00000057163

Image copying ==  000000455384.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000455384.jpg
Image copying ==  000000249352.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000249352.jpg
Image copying ==  000000348306.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000348306.jpg
Image copying ==  000000475593.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000475593.jpg
Image copying ==  000000145523.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000145523.jpg
Image copying ==  000000143370.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000143370.jpg
Image copying ==  000000279806.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000279806.jpg
Image copying ==  000000059171.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000059171.jpg
Image copying ==  000000369826.jpg
C:/Users/Vishal/Desktop/AIML/Capstone

Image copying ==  000000214274.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000214274.jpg
Image copying ==  000000385757.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000385757.jpg
Image copying ==  000000181586.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000181586.jpg
Image copying ==  000000383464.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000383464.jpg
Image copying ==  000000552979.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000552979.jpg
Image copying ==  000000289714.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000289714.jpg
Image copying ==  000000425690.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000425690.jpg
Image copying ==  000000019523.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000019523.jpg
Image copying ==  000000510992.jpg
C:/Users/Vishal/Desktop/AIML/Capstone

Image copying ==  000000281950.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000281950.jpg
Image copying ==  000000370210.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000370210.jpg
Image copying ==  000000531111.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000531111.jpg
Image copying ==  000000499177.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000499177.jpg
Image copying ==  000000075118.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000075118.jpg
Image copying ==  000000090732.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000090732.jpg
Image copying ==  000000301574.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000301574.jpg
Image copying ==  000000241876.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000241876.jpg
Image copying ==  000000034474.jpg
C:/Users/Vishal/Desktop/AIML/Capstone

Image copying ==  000000488943.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000488943.jpg
Image copying ==  000000396673.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000396673.jpg
Image copying ==  000000008218.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000008218.jpg
Image copying ==  000000140307.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000140307.jpg
Image copying ==  000000027065.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000027065.jpg
Image copying ==  000000264599.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000264599.jpg
Image copying ==  000000332653.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000332653.jpg
Image copying ==  000000239252.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000239252.jpg
Image copying ==  000000506906.jpg
C:/Users/Vishal/Desktop/AIML/Capstone

...

Image copying ==  000000368302.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000368302.jpg
Image copying ==  000000043561.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000043561.jpg
Image copying ==  000000128920.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000128920.jpg
Image copying ==  000000258350.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000258350.jpg
Image copying ==  000000412807.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000412807.jpg
Image copying ==  000000338452.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000338452.jpg
Image copying ==  000000299448.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000299448.jpg
Image copying ==  000000313627.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000313627.jpg
Image copying ==  000000137634.jpg
C:/Users/Vishal/Desktop/AIML/Capstone

Image copying ==  000000032682.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000032682.jpg
Image copying ==  000000053580.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000053580.jpg
Image copying ==  000000410932.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000410932.jpg
Image copying ==  000000001590.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000001590.jpg
Image copying ==  000000543577.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000543577.jpg
Image copying ==  000000035940.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000035940.jpg
Image copying ==  000000186322.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000186322.jpg
Image copying ==  000000289855.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000289855.jpg
Image copying ==  000000517623.jpg
C:/Users/Vishal/Desktop/AIML/Capstone

Image copying ==  000000010920.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000010920.jpg
Image copying ==  000000273728.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000273728.jpg
Image copying ==  000000018046.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000018046.jpg
Image copying ==  000000167964.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000167964.jpg
Image copying ==  000000209824.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000209824.jpg
Image copying ==  000000216327.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000216327.jpg
Image copying ==  000000268755.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000268755.jpg
Image copying ==  000000148392.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000148392.jpg
Image copying ==  000000375130.jpg
C:/Users/Vishal/Desktop/AIML/Capstone

Image copying ==  000000455435.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000455435.jpg
Image copying ==  000000232894.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000232894.jpg
Image copying ==  000000109924.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000109924.jpg
Image copying ==  000000075719.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000075719.jpg
Image copying ==  000000101145.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000101145.jpg
Image copying ==  000000406729.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000406729.jpg
Image copying ==  000000526143.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000526143.jpg
Image copying ==  000000158254.jpg
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/train2017/000000158254.jpg
Image copying ==  000000267812.jpg
C:/Users/Vishal/Desktop/AIML/Capstone

TypeError: can only concatenate str (not "float") to str

## To create GCC subset

In [0]:
with open(GCC_config,'r') as f:
            temp = f.readlines()
            f.close()

In [0]:
for x in temp:
        x2 = x.split('=')
        if x2[0].strip() == 'dataset':
            dataset = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'src_Caption_filename':
            src_Caption_filename = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'src_image_folder':
            src_image_folder = x2[1].strip()
            print(x2[1].strip())
        elif x2[0].strip() == 'destination_folder':
            destination_folder = x2[1].strip()
            print(x2[1].strip())

GCC
C:/Users/Vishal/Desktop/AIML/Capstone/Referrence/Dataset/ccSet.tsv
C:/Users/Vishal/Desktop/AIML/Capstone/Datasets/MSCOCO/subset8


In [0]:
counter = 0

In [0]:
if dataset == "fkr30k":
        caption_header = "comment"
        imagename_header = "image_name"
        count=8000
        flkr30k_process(data,count)
elif dataset == "fkr8k":
        caption_header = "comment"
        imagename_header = "image_name"
        count=8000
        flkr8k_process(data,count)
        #for data,count in dict.items():
        #    flkr8k_process(data,count)
elif dataset == "GCC":
        caption_header = "comment"
        imagename_header = "URL"
        count=2
       # for data,count in dict.items():
        GCC_process(count)
elif dataset == "mscoco":
        caption_header = "comment"
        imagename_header = "image_name"
        count=8000
        mscoco_process(data,count)

  portrait of young woman wearing glasses peeking behind the laptop while working and doing her business from home  \
0  a woman rests her cheek on her hand and looks ...                                                                 
1             a general view ahead of a book signing                                                                 
2  while it 's not known how much the house sold ...                                                                 
3      pylon and cables of the cable - stayed bridge                                                                 
4  sea salt in bowl and spoon on a wooden backgro...                                                                 

  http://l7.alamy.com/zooms/ece8bef20c8843ebaa636d23082e3dd4/portrait-of-young-woman-wearing-glasses-peeking-behind-the-laptop-g59m94.jpg  
0  https://media.gettyimages.com/photos/circa-195...                                                                                       
1  https://